In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'P2O5', 'l.i.', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
2809,72.408752,0.247289,14.669474,0.638732,1.782947,0.041314,0.689707,1.423317,3.952642,4.085334,0.060492
2810,72.292198,0.251972,14.706073,0.640730,1.809817,0.041577,0.712856,1.467313,3.952874,4.062925,0.061665
2811,72.253000,0.253433,14.721165,0.643486,1.814515,0.041632,0.722393,1.484052,3.951999,4.052965,0.061359
2812,72.218663,0.254218,14.735031,0.639248,1.823734,0.041633,0.731805,1.502799,3.949426,4.041972,0.061472
2813,72.175130,0.255858,14.749455,0.640096,1.832478,0.041710,0.741441,1.520710,3.948886,4.032574,0.061663


In [9]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
2809,72.408752,0.247289,14.669474,0.638732,1.782947,0.041314,0.689707,1.423317,3.952642,4.085334,0.060492
2810,72.292198,0.251972,14.706073,0.640730,1.809817,0.041577,0.712856,1.467313,3.952874,4.062925,0.061665
2811,72.253000,0.253433,14.721165,0.643486,1.814515,0.041632,0.722393,1.484052,3.951999,4.052965,0.061359
2812,72.218663,0.254218,14.735031,0.639248,1.823734,0.041633,0.731805,1.502799,3.949426,4.041972,0.061472
2813,72.175130,0.255858,14.749455,0.640096,1.832478,0.041710,0.741441,1.520710,3.948886,4.032574,0.061663
...,...,...,...,...,...,...,...,...,...,...,...
205097,72.268618,0.277036,14.504489,0.824244,1.811343,0.045136,0.594112,1.226596,4.070681,4.293169,0.084577
205541,71.908725,0.296794,14.617856,0.888130,1.860140,0.046280,0.655489,1.330200,4.080780,4.230478,0.085129
205542,72.002545,0.291713,14.589543,0.875398,1.843993,0.045987,0.639306,1.302024,4.078522,4.246520,0.084447
205543,72.105392,0.286649,14.554502,0.868704,1.822678,0.045723,0.619833,1.266532,4.078060,4.268313,0.083615


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:90000]

In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1/area1_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1/area1_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [13]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1/area1_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1/area1_2_results_excel.txt")

In [14]:
df_final = pd.concat([df_final1, df_final2])

In [15]:
df_final

,Q,P,K
SAMPLE,,,
2809,28.772768,36.764972,27.485120
2810,28.596781,36.977234,27.352893
2811,28.547932,37.055677,27.293357
2812,28.508049,37.128403,27.226182
2813,28.445591,37.211899,27.170181
...,...,...,...
205097,27.847736,36.530589,28.813148
205541,27.354191,37.117842,28.451252
205542,27.487501,36.965279,28.544120


## QAPF classification

In [16]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [17]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro', 'quartz monzonite'],
      dtype=object)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_ternary(df_final, a="Q", b="K", c="P", color="QAPF")

fig.write_image(f"../_FIGURES/geomap/qapf_ternary/area1_QAPF.pdf")
fig.show()

In [18]:
df_final.to_excel("../_CIPW/CIPW/AREA1/qapf.xlsx", index=True)

-------

In [19]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1/QAPF_counts.xlsx")

-----

In [34]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=None, usecols = ['area', 'Y_UTM', 'X_UTM', 'ZoneNumber', 'ZoneLetter'])

In [35]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,1.113927e+06,5.652652e+06,48.0,U,1.0,52.549507,17.912818,22.931202,monzo granite
2,1.068636e+06,5.598845e+06,48.0,U,1.0,45.940433,19.086384,32.357653,monzo granite
3,1.098840e+06,5.690263e+06,48.0,U,1.0,50.739986,6.408381,37.371890,syeno granite
4,9.048985e+05,5.630297e+06,48.0,U,1.0,41.407116,23.232361,27.590113,monzo granite
5,1.141240e+06,5.804930e+06,48.0,U,1.0,54.633030,17.945563,19.744849,monzo granite
...,...,...,...,...,...,...,...,...,...
4258,1.389363e+08,-2.395024e+08,48.0,U,5.0,24.564247,37.818520,16.856395,granodiorite
4259,1.163066e+08,-2.125908e+08,48.0,U,5.0,15.009478,45.257809,24.490858,quartz monzonite
4260,1.201955e+08,-2.279558e+08,48.0,U,5.0,19.211182,43.022987,16.844273,granodiorite
4261,1.191852e+08,-2.262132e+08,48.0,U,5.0,13.152993,53.142929,15.933668,quartz monzodiorite\nquartz monzogabbro


In [36]:
QAPF_control_area1 = QAPF_control[QAPF_control['area'] ==1]
QAPF_control_area1.to_excel("../_CIPW/CIPW/AREA1/QAPF_control_points.xlsx", index=True)